In [1]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

import sys
sys.path.insert(0, '/scratch/cinthiasouza/mv-text-summarizer')

from sumy.sumy.parsers.plaintext import PlaintextParser
from sumy.sumy.nlp.tokenizers import Tokenizer
from sumy.sumy.summarizers.lsa import LsaSummarizer as SummarizerLsa
from sumy.sumy.summarizers.lex_rank import LexRankSummarizer as SummarizerLex
from sumy.sumy.summarizers.sum_basic import SumBasicSummarizer as SummarizerSumBasic
from sumy.sumy.summarizers.text_rank import TextRankSummarizer  as SummarizerTextrank
from sumy.sumy.nlp.stemmers import Stemmer
from sumy.sumy.utils import get_stop_words

LANGUAGE = "english"
SENTENCES_COUNT=3

In [2]:
import re
import numpy as np
import pandas as pd
import pickle

In [9]:
def load_data(section):
    section='conclusion'

    with open('dataset/dataset_{}.pkl'.format('features'), 'rb') as fp:
        dataset = pickle.load(fp)
    
    train = dataset[section][4][['sentences', 'articles', 'rouge_1']]
    test = dataset[section][5][['sentences', 'articles', 'rouge_1']]
    
    return train, test

In [5]:
def summarizer_lsa(parser, stemmer):
    
    lsa = SummarizerLsa(stemmer)
    lsa.stop_words = get_stop_words(LANGUAGE)

    result = lsa(parser.document, SENTENCES_COUNT)
    
    return result

In [6]:
def summarizer_lex(parser, stemmer):
    
    lex = SummarizerLex(stemmer)
    lex.stop_words = get_stop_words(LANGUAGE)

    result = lex(parser.document, SENTENCES_COUNT)
    
    return result

In [7]:
def summarizer_text(parser, stemmer):
    
    text = SummarizerTextrank(stemmer)
    text.stop_words = get_stop_words(LANGUAGE)

    result = text(parser.document, SENTENCES_COUNT)
    
    return result

In [19]:
train, test = load_data(section='introduction')
articles_id = pd.unique(train['articles'])

test['sentences'] = test['sentences'].astype('str')

In [11]:
stemmer = Stemmer(LANGUAGE)

df_lsa = []
df_text = []
df_lex = []
    

for article_id in articles_id:

    aux = train.loc[train['articles'] == article_id]
    text = " ".join(aux['sentences'].tolist())
    
    parser = PlaintextParser(text, Tokenizer(LANGUAGE))
    df_lsa.append(summarizer_lsa(parser, stemmer))
    #df_text.append(summarizer_text(parser, stemmer))

In [18]:
train_lsa = pd.concat(df_lsa)